# Exploratory analysis of CA.txt real estate data

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%config InlineBackend.figure_format = 'retina'

In [2]:
import os
import time

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pylab as pl

import numpy as np
np.set_printoptions(precision=2, linewidth=120, suppress=True, edgeitems=4)

import pandas as pd
pd.set_option('display.max_columns', 150)
pd.set_option('display.max_rows', 50)
#pd.set_option('precision', 5)

In [3]:
from sklearn.model_selection import KFold,train_test_split,StratifiedKFold, GridSearchCV
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet, SGDRegressor,RandomizedLasso
from sklearn.feature_selection import RFE, f_regression, RFECV
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.svm import SVR


In [4]:
# os.chdir('/Users/zacklarsen/Dropbox/Datasets')
# !ls
# dropboxCAL = pd.read_table('CA.txt',sep='|',error_bad_lines=False)

# dropboxCAL.head(10)
# len(dropboxCAL.columns)

In [3]:
os.chdir('/Users/zacklarsen/Dropbox/Datasets/')
#!ls
CAL = pd.read_table('CA_3M.txt',sep='|',error_bad_lines=False,date_parser='RECORDINGDATE',nrows=10000)#  ,nrows=10000

//anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (13,72,106,109) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
CAL.columns

Index([u'FIPSCODE', u'PROPERTYUNIQUEID', u'APN', u'APNSEQUENCENBR',
       u'PREVIOUSPARCELNUMBER', u'TAXACCOUNTNUMBER', u'MatchedFlag',
       u'HOUSENUMBER', u'HOUSENUMBER2', u'HOUSENUMBERSUFFIX',
       ...
       u'SubdivisionName', u'PhaseNo', u'TractNo', u'SecTwnRngMer',
       u'AssessorsMapRef', u'LEGALDESCRIPTION', u'TITLECOMPANYNAME',
       u'MULTIAPN', u'UPDATETIMESTAMP', u'UCID'],
      dtype='object', length=136)

In [5]:
len(CAL['PROPERTYUNIQUEID'].unique())

6294

In [29]:
CAL = CAL.sort_values(by=['PROPERTYUNIQUEID','RECORDINGDATE'], ascending=[1,1])
CAL.loc[:,['PROPERTYUNIQUEID','RECORDINGDATE']].head(10)

,PROPERTYUNIQUEID,RECORDINGDATE
3853,9632957,2009-06-16
7798,9633057,2008-07-22
7164,9633057,2009-05-14
5240,9633297,2008-05-23
9272,9633300,2010-03-12
9273,9633300,2010-03-12
1531,9634058,2008-07-21
3671,9634147,2008-11-19
4334,9634151,2008-08-11
4335,9634151,2008-08-11


In [34]:
# Parse the recording date into a dateimte object
CAL['RECORDINGDATE'] = pd.to_datetime(CAL['RECORDINGDATE'], format='%Y%m%d')
CAL['SALEDATE'] = pd.to_datetime(CAL['SALEDATE'], format='%Y%m%d')
CAL['FIRSTMORTGAGERECORDINGDATE'] = pd.to_datetime(CAL['FIRSTMORTGAGERECORDINGDATE'], format='%Y%m%d')
CAL['FIRSTMORTGAGEDOCDATE'] = pd.to_datetime(CAL['FIRSTMORTGAGEDOCDATE'], format='%Y%m%d')
CAL['FIRSTMORTGAGEDUEDATE'] = pd.to_datetime(CAL['FIRSTMORTGAGEDUEDATE'], format='%Y%m%d')

#CAL['RECORDINGDATE'].head(5)

In [35]:
CAL.head(20)
len(CAL.columns)

,FIPSCODE,PROPERTYUNIQUEID,APN,MatchedFlag,HOUSENUMBER,DIRECTIONLEFT,STREETNAME,Suffix,DIRECTIONRIGHT,UNITTYPE,UNITNUMBER,CITY,STATE,ZIPCODE,ZIPPLUS4,CARRIERCODE,CORPORATEINDICATOR,BUYERNAMEBORROWERNAME1,OWNERETALIND,OwnerCAREOfNAME,OWNERSHIPRIGHTSCODE1,OWNERRELATIONSHIPTYPE1,CORPORATEINDICATOR2,BUYERNAMEBORROWERNAME2,OWNERRELATIONSHIPTYPE2,BUYER1LASTNAME,BUYER1FIRSTNAME,BUYER1MIDDLEName,BUYER1SUFFIX,BUYER2LASTNAME,BUYER2FIRSTNAME,BUYER2MIDDLEName,BUYER2SUFFIX,MAILHOUSENUMBER,MAILDIRECTIONLEFT,MAILSTREETNAME,MAILMODE,MAILDIRECTIONRIGHT,MAILUNITTYPE,MAILUNITNUMBER,MAILCITY,MAILSTATE,MAILZIPCODE,MAILZIPPlus4,MAILCARRIERCODE,FirstAmericanUniqueTransactionID,RECORDINGDATE,SALEDATE,DOCUMENTNUMBER,DOCUMENTTYPE,TRANSACTIONTYPE,SELLERNAME,SELLERNAMECORPINDICATOR,SELLERNAME2,SELLERNAME2CORPINDICATOR,SALEAMOUNT,TRANSFERTAX,PARTIALINTERESTINDICATORFLAG,RESALEFLAG,NEWCONSTRUCTIONFLAG,INTERFAMILYFLAG,FORCLOSUREFLAG,REOSALEFLAG,FIRSTMORTGAGERECORDINGDATE,FIRSTMORTGAGEDOCDATE,FIRSTMORTGAGEDOCUMENTNUMBER,FIRSTMORTGAGEDETAILEDDOCUMENTTYPE,FIRSTMORTGAGEMORTGAGEAMOUNT,FIRSTMORTGAGELENDERNAME,FIRSTMORTGAGELOANTYPECODE,FIRSTMORTGAGETERMCODE,FIRSTMORTGAGETERM,FIRSTMORTGAGEDUEDATE,FIRSTMORTGAGEINTERESTRATETYPE,FIRSTMORTGAGEINTERESTRATE,FIRSTMORTGAGESELLERCARRYBACKFLAG,FIRSTMORTGAGEPRIVATEPARTYLENDER,FIRSTMORTGAGECONSTRUCTIONLOAN,FIRSTMORTGAGEOTHERSUBORDINATELOANFlag,SECONDMORTGAGERECORDINGDATE,SECONDMORTGAGEDOCDATE,SECONDMORTGAGEDOCUMENTNUMBER,SECONDMORTGAGEDETAILEDDOCUMENTTYPE,SECONDMORTGAGEAMOUNT,SECONDMORTGAGELENDERNAME,SECONDMORTGAGELOANTYPECODE,SECONDMORTGAGETERMCODE,SECONDMORTGAGETERM,SECONDMORTGAGEDUEDATE,SECONDMORTGAGEINTERESTRATETYPE,SECONDMORTGAGEINTERESTRATE,SECONDMORTGAGEEQUITYFLAG,SECONDMORTGAGEOTHERSUBORDINATELOANFlag,LotCode,LandLot,Block,Unit,CityMunicipalityTownship,SubdivisionName,TractNo,SecTwnRngMer,LEGALDESCRIPTION,TITLECOMPANYNAME,MULTIAPN,UPDATETIMESTAMP,UCID
3853,6001,9632957,10-817-2,Y,587,NaN,VERNON,ST,NaN,NaN,NaN,OAKLAND,CA,94610.0,1434.0,C005,Y,FEDERAL HOME LOAN MORTGAGE CORPORATION,NaN,NaN,NaN,BE,T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1021777529,2009-06-16,2009-06-09,2009189766,69,4,PARKER MELISSA,NaN,NaN,NaN,341238,0,NaN,NaN,NaN,NaN,Y,NaN,NaT,NaT,NaN,NaN,0,NaN,NaN,NaN,NaN,NaT,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,D,NaN,OAKLAND,THE STANFORD TRACT,NaN,NaN,NaN,NORTH AMERICAN TITLE COMPANY,NaN,20160927,U
7798,6001,9633057,10-818-22,Y,520,NaN,OAKLAND,AVE,NaN,NaN,NaN,OAKLAND,CA,94611.0,5429.0,C038,Y,WELLS FARGO BANK NA,NaN,GMAC MORTGAGE LLC,NaN,TR,T,NaN,BE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1100,NaN,VIRGINIA,DR,NaN,NaN,NaN,FORT WASHINGTON,PA,19034.0,3204.0,C008,1023128992,2008-07-22,2008-06-27,2008222307,69,4,CHANDLER PATRICIA,NaN,CHANDLER WILLIAM,NaN,439677,0,NaN,NaN,NaN,NaN,Y,NaN,NaT,NaT,NaN,NaN,0,NaN,NaN,NaN,NaN,NaT,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,P,1,NaN,NaN,OAKLAND,SANTA CLARA AVENUE PROPERTY,NaN,NaN,NORTHEASTERN37 FT LOT1,FATCO,NaN,20160927,U
7164,6001,9633057,10-818-22,Y,520,NaN,OAKLAND,AVE,NaN,NaN,NaN,OAKLAND,CA,94611.0,5429.0,C038,NaN,MARCUS ANTHONY,NaN,NaN,CP,HW,NaN,HAAN PATRICE M,HW,MARCUS,ANTHONY,NaN,NaN,HAAN,PATRICE,M,NaN,2644,NaN,WAKEFIELD,AVE,NaN,NaN,NaN,OAKLAND,CA,94606.0,3554.0,C024,1023128995,2009-05-14,2009-04-21,2009153432,36,2,WELLS FARGO BANK N A,Y,NaN,NaN,350000,38500,NaN,Y,NaN,NaN,NaN,Y,NaT,NaT,NaN,NaN,0,NaN,NaN,NaN,NaN,NaT,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,P,1,NaN,NaN,OAKLAND,SANTA CLARA AVENUE PROPERTY,NaN,NaN,NaN,FIRST AMERICAN TITLE COMPANY,NaN,20160927,U
5240,6001,9633297,10-821-7,Y,480,NaN,CHETWOOD,ST,NaN,NaN,NaN,OAKLAND,CA,94610.0,2649.0,C005,NaN,ESTABROOK REED,NaN,NaN,JT,HW,NaN,ESTABROOK GABRIELE,HW,ESTABROOK,REED,NaN,NaN,ESTABROOK,GABRIELE,NaN,NaN,480,NaN,CHETWOOD,ST,NaN,NaN,NaN,OAKLAND,CA,94610.0,2649.0,C005,1022868367,2008-05-23,2008-05-02,2008167754,40,1,ESTABROOK REED,NaN,NaN,NaN,32500,3575,NaN,NaN,NaN,NaN,Na

106

In [37]:
# Retrieve dataframe rows for the most frequently occurring PROPERTYUNIQUEID

CAL[CAL['PROPERTYUNIQUEID'] == CAL['PROPERTYUNIQUEID'].value_counts().idxmax()]

,FIPSCODE,PROPERTYUNIQUEID,APN,MatchedFlag,HOUSENUMBER,DIRECTIONLEFT,STREETNAME,Suffix,DIRECTIONRIGHT,UNITTYPE,UNITNUMBER,CITY,STATE,ZIPCODE,ZIPPLUS4,CARRIERCODE,CORPORATEINDICATOR,BUYERNAMEBORROWERNAME1,OWNERETALIND,OwnerCAREOfNAME,OWNERSHIPRIGHTSCODE1,OWNERRELATIONSHIPTYPE1,CORPORATEINDICATOR2,BUYERNAMEBORROWERNAME2,OWNERRELATIONSHIPTYPE2,BUYER1LASTNAME,BUYER1FIRSTNAME,BUYER1MIDDLEName,BUYER1SUFFIX,BUYER2LASTNAME,BUYER2FIRSTNAME,BUYER2MIDDLEName,BUYER2SUFFIX,MAILHOUSENUMBER,MAILDIRECTIONLEFT,MAILSTREETNAME,MAILMODE,MAILDIRECTIONRIGHT,MAILUNITTYPE,MAILUNITNUMBER,MAILCITY,MAILSTATE,MAILZIPCODE,MAILZIPPlus4,MAILCARRIERCODE,FirstAmericanUniqueTransactionID,RECORDINGDATE,SALEDATE,DOCUMENTNUMBER,DOCUMENTTYPE,TRANSACTIONTYPE,SELLERNAME,SELLERNAMECORPINDICATOR,SELLERNAME2,SELLERNAME2CORPINDICATOR,SALEAMOUNT,TRANSFERTAX,PARTIALINTERESTINDICATORFLAG,RESALEFLAG,NEWCONSTRUCTIONFLAG,INTERFAMILYFLAG,FORCLOSUREFLAG,REOSALEFLAG,FIRSTMORTGAGERECORDINGDATE,FIRSTMORTGAGEDOCDATE,FIRSTMORTGAGEDOCUMENTNUMBER,FIRSTMORTGAGEDETAILEDDOCUMENTTYPE,FIRSTMORTGAGEMORTGAGEAMOUNT,FIRSTMORTGAGELENDERNAME,FIRSTMORTGAGELOANTYPECODE,FIRSTMORTGAGETERMCODE,FIRSTMORTGAGETERM,FIRSTMORTGAGEDUEDATE,FIRSTMORTGAGEINTERESTRATETYPE,FIRSTMORTGAGEINTERESTRATE,FIRSTMORTGAGESELLERCARRYBACKFLAG,FIRSTMORTGAGEPRIVATEPARTYLENDER,FIRSTMORTGAGECONSTRUCTIONLOAN,FIRSTMORTGAGEOTHERSUBORDINATELOANFlag,SECONDMORTGAGERECORDINGDATE,SECONDMORTGAGEDOCDATE,SECONDMORTGAGEDOCUMENTNUMBER,SECONDMORTGAGEDETAILEDDOCUMENTTYPE,SECONDMORTGAGEAMOUNT,SECONDMORTGAGELENDERNAME,SECONDMORTGAGELOANTYPECODE,SECONDMORTGAGETERMCODE,SECONDMORTGAGETERM,SECONDMORTGAGEDUEDATE,SECONDMORTGAGEINTERESTRATETYPE,SECONDMORTGAGEINTERESTRATE,SECONDMORTGAGEEQUITYFLAG,SECONDMORTGAGEOTHERSUBORDINATELOANFlag,LotCode,LandLot,Block,Unit,CityMunicipalityTownship,SubdivisionName,TractNo,SecTwnRngMer,LEGALDESCRIPTION,TITLECOMPANYNAME,MULTIAPN,UPDATETIMESTAMP,UCID
3109,6001,9738778,444-48-16,Y,25016,NaN,LUCIEN,WAY,NaN,NaN,NaN,HAYWARD,CA,94544.0,1911.0,C040,NaN,TIZON DANILO,NaN,NaN,JT,HW,NaN,JAVIER VIVIAN,HW,TIZON,DANILO,NaN,NaN,JAVIER,VIVIAN,NaN,NaN,24960,NaN,ARBOR CREST,CIR,NaN,NaN,NaN,HAYWARD,CA,94544.0,1232.0,C040,1021589877,2001-03-16,2001-03-14,2001091921,36,3,GREYSTONE HOMES INC,Y,LENNAR SALES CORP,Y,397500,43725,NaN,NaN,Y,NaN,NaN,NaN,2001-03-16,2001-03-14,2.001092e+09,MG,317850,GREENPOINT MTG FUNDING INC,NaN,M,360.0,2031-04-01,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN,60,NaN,NaN,HAYWARD,NaN,7239,NaN,NaN,NORTH AMERICAN TITLE CO,1.0,20161111,U
6910,6001,9738778,444-48-16,Y,25016,NaN,LUCIEN,WAY,NaN,NaN,NaN,HAYWARD,CA,94544.0,1911.0,C040,NaN,LACUSTA ROMMEL,NaN,NaN,JT,HW,NaN,MARIANO LEAH ROSE L,HW,LACUSTA,ROMMEL,NaN,V,MARIANO,LEAH,ROSE L,NaN,24968,NaN,ARBOR CREST,CIR,NaN,NaN,NaN,HAYWARD,CA,94544.0,1232.0,C040,1009944836,2001-03-16,2001-03-08,2001091872,36,3,GREYSTONE HOMES INC,Y,LENNAR SALES CORP,Y,419500,46145,NaN,NaN,Y,NaN,NaN,NaN,2001-03-16,2001-03-08,2.001092e+09,MG,335500,UNIVERSAL AMERICAN MTG COMPANY OF CA,NaN,M,360.0,2031-04-01,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN,59,NaN,NaN,HAYWARD,NaN,7239,NaN,NaN,NORTH AMERICAN TITLE CO,1.0,20161111,U
2616,6001,9738778,444-48-16,Y,25016,NaN,LUCIEN,WAY,NaN,NaN,NaN,HAYWARD,CA,94544.0,1911.0,C040,NaN,WONG MELVIN,NaN,NaN,JT,SM,NaN,KWONG LISA S,SW,WONG,MELVIN,NaN,NaN,KWONG,LISA,S,NaN,24976,NaN,ARBOR CREST,CIR,NaN,NaN,NaN,HAYWARD,CA,94544.0,1232.0,C040,1009944809,2001-03-29,2001-03-14,2001107542,36,3,GREYSTONE HOMES INC,Y,LENNAR SALES CORP,Y,404500,44495,NaN,NaN,Y,NaN,NaN,NaN,2001-03-29,2001-03-14,2.001108e+09,MG,323400,UNIVERSAL AMERICAN MTG COMPANY OF CA,NaN,M,360.0,2031-04-01,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,68,NaN,NaN,HAYWARD,NaN,7239,NaN,NaN,NORTH AMERICAN TITLE CO,1.0,20161111,U
4734,6001,9738778,444-48-16,Y,25016,NaN,LUCIEN,WAY,NaN,NaN,NaN,HAYWARD,CA,94544.0,1911.0,C040,NaN,WU YONGLI,NaN,NaN,MM,MM,NaN,NaN,NaN,WU,YONGLI,NaN,NaN,NaN,NaN,NaN,NaN,24967,

In [49]:
# Calculate the recording date - sale date

CAL['RECORDINGDATE'].sub(CAL['SALEDATE'],axis=0)

3853     7 days
7798    25 days
7164    23 days
5240    21 days
9272     7 days
9273     7 days
1531    14 days
3671     6 days
4334     2 days
4335     2 days
9307    63 days
8412    13 days
7118    46 days
9271     2 days
1439    22 days
8033     5 days
4966    14 days
8549   174 days
1749     8 days
6997     4 days
8504    13 days
5661     5 days
7596    48 days
8622     1 days
8623     9 days
         ...   
9892     9 days
6932     2 days
1690    14 days
7180     5 days
7692    75 days
8744     1 days
1126    12 days
3038     3 days
1068    26 days
2751        NaT
1521    15 days
4449    10 days
8037    10 days
6475     1 days
9509    26 days
1146    46 days
5355     2 days
5664     2 days
5869     5 days
5870     5 days
1589     6 days
7621     2 days
7623     2 days
9944    64 days
9943    64 days
dtype: timedelta64[ns]

In [ ]:
# These columns are completely Na values
'''
'APNSEQUENCENBR', 
'PREVIOUSPARCELNUMBER', 
'HOUSENUMBER2',
'HOUSENUMBERSUFFIX',
'MAILHOUSENUMBER2', 
'MAILHOUSENUMBERSUFFIX',
'HAWAIILANDCOURTDOCUMENTNUMBER',
'HAWAIITCTCERTIFICATENUMBER',
'SECONDMORTGAGESELLERCARRYBACK',
'SECONDMORTGAGEPRIVATEPARTYLENDER',
'SECONDMORTGAGECONSTRUCTIONLOAN',
'SECONDMORTGAGEREFIFLAG',
'SECONDMORTGAGEHELOCFLAG'
'''

In [8]:
misslist = CAL.columns[CAL.isnull().all()] # These are the columns with all missing values

CAL = CAL.drop(misslist,axis=1)

len(CAL.columns) # Should be reduced from the original 136 to 123

106

In [9]:
CAL.head(4)
len(CAL)

,FIPSCODE,PROPERTYUNIQUEID,APN,MatchedFlag,HOUSENUMBER,DIRECTIONLEFT,STREETNAME,Suffix,DIRECTIONRIGHT,UNITTYPE,UNITNUMBER,CITY,STATE,ZIPCODE,ZIPPLUS4,CARRIERCODE,CORPORATEINDICATOR,BUYERNAMEBORROWERNAME1,OWNERETALIND,OwnerCAREOfNAME,OWNERSHIPRIGHTSCODE1,OWNERRELATIONSHIPTYPE1,CORPORATEINDICATOR2,BUYERNAMEBORROWERNAME2,OWNERRELATIONSHIPTYPE2,BUYER1LASTNAME,BUYER1FIRSTNAME,BUYER1MIDDLEName,BUYER1SUFFIX,BUYER2LASTNAME,BUYER2FIRSTNAME,BUYER2MIDDLEName,BUYER2SUFFIX,MAILHOUSENUMBER,MAILDIRECTIONLEFT,MAILSTREETNAME,MAILMODE,MAILDIRECTIONRIGHT,MAILUNITTYPE,MAILUNITNUMBER,MAILCITY,MAILSTATE,MAILZIPCODE,MAILZIPPlus4,MAILCARRIERCODE,FirstAmericanUniqueTransactionID,RECORDINGDATE,SALEDATE,DOCUMENTNUMBER,DOCUMENTTYPE,TRANSACTIONTYPE,SELLERNAME,SELLERNAMECORPINDICATOR,SELLERNAME2,SELLERNAME2CORPINDICATOR,SALEAMOUNT,TRANSFERTAX,PARTIALINTERESTINDICATORFLAG,RESALEFLAG,NEWCONSTRUCTIONFLAG,INTERFAMILYFLAG,FORCLOSUREFLAG,REOSALEFLAG,FIRSTMORTGAGERECORDINGDATE,FIRSTMORTGAGEDOCDATE,FIRSTMORTGAGEDOCUMENTNUMBER,FIRSTMORTGAGEDETAILEDDOCUMENTTYPE,FIRSTMORTGAGEMORTGAGEAMOUNT,FIRSTMORTGAGELENDERNAME,FIRSTMORTGAGELOANTYPECODE,FIRSTMORTGAGETERMCODE,FIRSTMORTGAGETERM,FIRSTMORTGAGEDUEDATE,FIRSTMORTGAGEINTERESTRATETYPE,FIRSTMORTGAGEINTERESTRATE,FIRSTMORTGAGESELLERCARRYBACKFLAG,FIRSTMORTGAGEPRIVATEPARTYLENDER,FIRSTMORTGAGECONSTRUCTIONLOAN,FIRSTMORTGAGEOTHERSUBORDINATELOANFlag,SECONDMORTGAGERECORDINGDATE,SECONDMORTGAGEDOCDATE,SECONDMORTGAGEDOCUMENTNUMBER,SECONDMORTGAGEDETAILEDDOCUMENTTYPE,SECONDMORTGAGEAMOUNT,SECONDMORTGAGELENDERNAME,SECONDMORTGAGELOANTYPECODE,SECONDMORTGAGETERMCODE,SECONDMORTGAGETERM,SECONDMORTGAGEDUEDATE,SECONDMORTGAGEINTERESTRATETYPE,SECONDMORTGAGEINTERESTRATE,SECONDMORTGAGEEQUITYFLAG,SECONDMORTGAGEOTHERSUBORDINATELOANFlag,LotCode,LandLot,Block,Unit,CityMunicipalityTownship,SubdivisionName,TractNo,SecTwnRngMer,LEGALDESCRIPTION,TITLECOMPANYNAME,MULTIAPN,UPDATETIMESTAMP,UCID
3853,6001,9632957,10-817-2,Y,587,NaN,VERNON,ST,NaN,NaN,NaN,OAKLAND,CA,94610.0,1434.0,C005,Y,FEDERAL HOME LOAN MORTGAGE CORPORATION,NaN,NaN,NaN,BE,T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1021777529,20090616,20090609.0,2009189766,69,4,PARKER MELISSA,NaN,NaN,NaN,341238,0,NaN,NaN,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,D,NaN,OAKLAND,THE STANFORD TRACT,NaN,NaN,NaN,NORTH AMERICAN TITLE COMPANY,NaN,20160927,U
7798,6001,9633057,10-818-22,Y,520,NaN,OAKLAND,AVE,NaN,NaN,NaN,OAKLAND,CA,94611.0,5429.0,C038,Y,WELLS FARGO BANK NA,NaN,GMAC MORTGAGE LLC,NaN,TR,T,NaN,BE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1100,NaN,VIRGINIA,DR,NaN,NaN,NaN,FORT WASHINGTON,PA,19034.0,3204.0,C008,1023128992,20080722,20080627.0,2008222307,69,4,CHANDLER PATRICIA,NaN,CHANDLER WILLIAM,NaN,439677,0,NaN,NaN,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,P,1,NaN,NaN,OAKLAND,SANTA CLARA AVENUE PROPERTY,NaN,NaN,NORTHEASTERN37 FT LOT1,FATCO,NaN,20160927,U
7164,6001,9633057,10-818-22,Y,520,NaN,OAKLAND,AVE,NaN,NaN,NaN,OAKLAND,CA,94611.0,5429.0,C038,NaN,MARCUS ANTHONY,NaN,NaN,CP,HW,NaN,HAAN PATRICE M,HW,MARCUS,ANTHONY,NaN,NaN,HAAN,PATRICE,M,NaN,2644,NaN,WAKEFIELD,AVE,NaN,NaN,NaN,OAKLAND,CA,94606.0,3554.0,C024,1023128995,20090514,20090421.0,2009153432,36,2,WELLS FARGO BANK N A,Y,NaN,NaN,350000,38500,NaN,Y,NaN,NaN,NaN,Y,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,P,1,NaN,NaN,OAKLAND,SANTA CLARA AVENUE PROPERTY,NaN,NaN,NaN,FIRST AMERICAN TITLE COMPANY,NaN,20160927,U
5240,6001,9633297,10-821-7,Y,480,NaN,CHETWOOD,ST,NaN,NaN,NaN,OAKLAND,CA,94610.0,2649.0,C005,NaN,ESTABROOK REED,NaN,NaN,JT,HW,NaN,ESTABROOK GABRIELE,HW,ESTABROOK,REED,NaN,NaN,ESTABROOK,GABRIELE,NaN,NaN,480,NaN,CHETWOOD,ST,NaN,NaN,NaN,OAKLAND,CA,94610.0,2649.0,C005,1022868367,20080523,20080502.0,2008167754,40,1,ESTABROOK REED,NaN,NaN,NaN,32500,3575,NaN,NaN,NaN,NaN,NaN,NaN,20

10000

In [10]:
nadrop = CAL.dropna(axis=1) # Drop any columns that have ANY Na values
nadrop.head(3)

,FIPSCODE,PROPERTYUNIQUEID,APN,STREETNAME,STATE,BUYERNAMEBORROWERNAME1,FirstAmericanUniqueTransactionID,RECORDINGDATE,DOCUMENTNUMBER,DOCUMENTTYPE,TRANSACTIONTYPE,SELLERNAME,SALEAMOUNT,TRANSFERTAX,FIRSTMORTGAGEMORTGAGEAMOUNT,SECONDMORTGAGEAMOUNT,UPDATETIMESTAMP,UCID
3853,6001,9632957,10-817-2,VERNON,CA,FEDERAL HOME LOAN MORTGAGE CORPORATION,1021777529,20090616,2009189766,69,4,PARKER MELISSA,341238,0,0,0,20160927,U
7798,6001,9633057,10-818-22,OAKLAND,CA,WELLS FARGO BANK NA,1023128992,20080722,2008222307,69,4,CHANDLER PATRICIA,439677,0,0,0,20160927,U
7164,6001,9633057,10-818-22,OAKLAND,CA,MARCUS ANTHONY,1023128995,20090514,2009153432,36,2,WELLS FARGO BANK N A,350000,38500,0,0,20160927,U


In [39]:
# Now, let's see if we can create a dataframe with no missing values for the following columns
# that Stephen said were important:

toplist = [
    'SALEDATE',
    'FIRSTMORTGAGEINTERESTRATE',
    'OWNERRELATIONSHIPTYPE1',
    'OWNERRELATIONSHIPTYPE2',
    'TRANSACTIONTYPE',
    'FIRSTMORTGAGEMORTGAGEAMOUNT',
    'HOUSENUMBER',
    'STREETNAME',
    'ZIPCODE',
    'CITY',
    'APN',
    'BUYERNAMEBORROWERNAME1',
    'RECORDINGDATE',
    'STATE'    
]

CALtop13 = CAL.dropna(subset = toplist)
len(CALtop13) # There are 1,350,119 rows in this DF

# Verify with:
# CALtop13[CALtop13['STATE'].notnull()]

5007

In [40]:
CALtop13[~CALtop13['DIRECTIONLEFT'].isnull()]

,FIPSCODE,PROPERTYUNIQUEID,APN,MatchedFlag,HOUSENUMBER,DIRECTIONLEFT,STREETNAME,Suffix,DIRECTIONRIGHT,UNITTYPE,UNITNUMBER,CITY,STATE,ZIPCODE,ZIPPLUS4,CARRIERCODE,CORPORATEINDICATOR,BUYERNAMEBORROWERNAME1,OWNERETALIND,OwnerCAREOfNAME,OWNERSHIPRIGHTSCODE1,OWNERRELATIONSHIPTYPE1,CORPORATEINDICATOR2,BUYERNAMEBORROWERNAME2,OWNERRELATIONSHIPTYPE2,BUYER1LASTNAME,BUYER1FIRSTNAME,BUYER1MIDDLEName,BUYER1SUFFIX,BUYER2LASTNAME,BUYER2FIRSTNAME,BUYER2MIDDLEName,BUYER2SUFFIX,MAILHOUSENUMBER,MAILDIRECTIONLEFT,MAILSTREETNAME,MAILMODE,MAILDIRECTIONRIGHT,MAILUNITTYPE,MAILUNITNUMBER,MAILCITY,MAILSTATE,MAILZIPCODE,MAILZIPPlus4,MAILCARRIERCODE,FirstAmericanUniqueTransactionID,RECORDINGDATE,SALEDATE,DOCUMENTNUMBER,DOCUMENTTYPE,TRANSACTIONTYPE,SELLERNAME,SELLERNAMECORPINDICATOR,SELLERNAME2,SELLERNAME2CORPINDICATOR,SALEAMOUNT,TRANSFERTAX,PARTIALINTERESTINDICATORFLAG,RESALEFLAG,NEWCONSTRUCTIONFLAG,INTERFAMILYFLAG,FORCLOSUREFLAG,REOSALEFLAG,FIRSTMORTGAGERECORDINGDATE,FIRSTMORTGAGEDOCDATE,FIRSTMORTGAGEDOCUMENTNUMBER,FIRSTMORTGAGEDETAILEDDOCUMENTTYPE,FIRSTMORTGAGEMORTGAGEAMOUNT,FIRSTMORTGAGELENDERNAME,FIRSTMORTGAGELOANTYPECODE,FIRSTMORTGAGETERMCODE,FIRSTMORTGAGETERM,FIRSTMORTGAGEDUEDATE,FIRSTMORTGAGEINTERESTRATETYPE,FIRSTMORTGAGEINTERESTRATE,FIRSTMORTGAGESELLERCARRYBACKFLAG,FIRSTMORTGAGEPRIVATEPARTYLENDER,FIRSTMORTGAGECONSTRUCTIONLOAN,FIRSTMORTGAGEOTHERSUBORDINATELOANFlag,SECONDMORTGAGERECORDINGDATE,SECONDMORTGAGEDOCDATE,SECONDMORTGAGEDOCUMENTNUMBER,SECONDMORTGAGEDETAILEDDOCUMENTTYPE,SECONDMORTGAGEAMOUNT,SECONDMORTGAGELENDERNAME,SECONDMORTGAGELOANTYPECODE,SECONDMORTGAGETERMCODE,SECONDMORTGAGETERM,SECONDMORTGAGEDUEDATE,SECONDMORTGAGEINTERESTRATETYPE,SECONDMORTGAGEINTERESTRATE,SECONDMORTGAGEEQUITYFLAG,SECONDMORTGAGEOTHERSUBORDINATELOANFlag,LotCode,LandLot,Block,Unit,CityMunicipalityTownship,SubdivisionName,TractNo,SecTwnRngMer,LEGALDESCRIPTION,TITLECOMPANYNAME,MULTIAPN,UPDATETIMESTAMP,UCID
6350,6001,9647171,12-964-11,Y,678,W,MACARTHUR,BLVD,NaN,NaN,NaN,OAKLAND,CA,94609.0,2340.0,C033,Y,HSBC BANK USA NA,NaN,NaN,TR,TR,NaN,NaN,BE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PO BOX,NaN,NaN,NaN,NaN,BUFFALO,NY,21045.0,NaN,NaN,1010392016,2008-07-30,2008-07-26,2008230292,69,4,TRUONG HUNG THANH,NaN,NaN,NaN,393750,0,NaN,NaN,NaN,NaN,Y,NaN,NaT,NaT,NaN,NaN,0,NaN,NaN,NaN,NaN,NaT,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,P,3,C,NaN,OAKLAND,APGAR TRACT,NaN,NaN,EASTERN40 FT LOT3 POR PLOT #37 KELLERSBERGERS ...,NONE AVAILABLE,NaN,20160927,U
5710,6001,9647171,12-964-11,Y,678,W,MACARTHUR,BLVD,NaN,NaN,NaN,OAKLAND,CA,94609.0,2340.0,C033,NaN,BISTA NGAWANG THARCHEN,NaN,NaN,TC,MM,NaN,BISTA NGAWANG TSULTIM,MM,BISTA,NGAWANG,THARCHEN,NaN,BISTA,NGAWANG,TSULTIM,NaN,678,W,MACARTHUR,BLVD,NaN,NaN,NaN,OAKLAND,CA,94609.0,2340.0,C033,1010392019,2009-04-13,2009-03-30,2009107148,36,2,HSBC BANK USA NATIONAL ASSOCIATION,Y,NaN,NaN,335000,36850,NaN,Y,NaN,NaN,NaN,Y,NaT,NaT,NaN,NaN,0,NaN,NaN,NaN,NaN,NaT,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,P,3,C,NaN,OAKLAND,APGAR TRACT,NaN,NaN,NaN,STEWART TITLE,NaN,20160927,U
2871,6001,9647189,12-964-16,Y,704,W,MACARTHUR,BLVD,NaN,NaN,NaN,OAKLAND,CA,94609.0,2326.0,C033,NaN,THATHIAH DAVID D,NaN,NaN,CP,HW,NaN,THANTHIAH VASANTHEE,HW,THATHIAH,DAVID,D,NaN,THANTHIAH,VASANTHEE,NaN,NaN,884,N,MAYFAIR,AVE,NaN,NaN,NaN,DALY CITY,CA,94015.0,3053.0,C019,1010392043,2008-12-24,2008-12-19,2008358888,36,2,LE KENT PHU,NaN,NaN,NaN,260000,28600,NaN,Y,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,0,NaN,NaN,NaN,NaN,NaT,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MP,24&23,C,NaN,OAKLAND,MAJOR GIVEN TRACT,NaN,NaN,NaN,FIDELITY NATIONAL TITLE CO,NaN,20160927,U
6386,6001,9647406,12-965-28,Y,705,W,MACARTHUR,BLVD,NaN,NaN,NaN,OAKLAND,CA,94609.0,2325.0,C033,NaN,GEBRESELASSIE YIKAALO,NaN,NaN,LV,TR,NaN,NaN,LV,GEBRESELASSIE,YIKAALO,NaN,NaN,NaN,NaN,NaN,NaN,1825,NaN,POGGI,ST,NaN,NaN,NaN,ALAMEDA,CA,94501.0,1853.0,C028,1020471787,2009-07-28,2009-07-06,2009241424,40,1,GEBRESELASSIE YIKAALO,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN

In [41]:
CALtop13[~CALtop13['UNITTYPE'].isnull()]

,FIPSCODE,PROPERTYUNIQUEID,APN,MatchedFlag,HOUSENUMBER,DIRECTIONLEFT,STREETNAME,Suffix,DIRECTIONRIGHT,UNITTYPE,UNITNUMBER,CITY,STATE,ZIPCODE,ZIPPLUS4,CARRIERCODE,CORPORATEINDICATOR,BUYERNAMEBORROWERNAME1,OWNERETALIND,OwnerCAREOfNAME,OWNERSHIPRIGHTSCODE1,OWNERRELATIONSHIPTYPE1,CORPORATEINDICATOR2,BUYERNAMEBORROWERNAME2,OWNERRELATIONSHIPTYPE2,BUYER1LASTNAME,BUYER1FIRSTNAME,BUYER1MIDDLEName,BUYER1SUFFIX,BUYER2LASTNAME,BUYER2FIRSTNAME,BUYER2MIDDLEName,BUYER2SUFFIX,MAILHOUSENUMBER,MAILDIRECTIONLEFT,MAILSTREETNAME,MAILMODE,MAILDIRECTIONRIGHT,MAILUNITTYPE,MAILUNITNUMBER,MAILCITY,MAILSTATE,MAILZIPCODE,MAILZIPPlus4,MAILCARRIERCODE,FirstAmericanUniqueTransactionID,RECORDINGDATE,SALEDATE,DOCUMENTNUMBER,DOCUMENTTYPE,TRANSACTIONTYPE,SELLERNAME,SELLERNAMECORPINDICATOR,SELLERNAME2,SELLERNAME2CORPINDICATOR,SALEAMOUNT,TRANSFERTAX,PARTIALINTERESTINDICATORFLAG,RESALEFLAG,NEWCONSTRUCTIONFLAG,INTERFAMILYFLAG,FORCLOSUREFLAG,REOSALEFLAG,FIRSTMORTGAGERECORDINGDATE,FIRSTMORTGAGEDOCDATE,FIRSTMORTGAGEDOCUMENTNUMBER,FIRSTMORTGAGEDETAILEDDOCUMENTTYPE,FIRSTMORTGAGEMORTGAGEAMOUNT,FIRSTMORTGAGELENDERNAME,FIRSTMORTGAGELOANTYPECODE,FIRSTMORTGAGETERMCODE,FIRSTMORTGAGETERM,FIRSTMORTGAGEDUEDATE,FIRSTMORTGAGEINTERESTRATETYPE,FIRSTMORTGAGEINTERESTRATE,FIRSTMORTGAGESELLERCARRYBACKFLAG,FIRSTMORTGAGEPRIVATEPARTYLENDER,FIRSTMORTGAGECONSTRUCTIONLOAN,FIRSTMORTGAGEOTHERSUBORDINATELOANFlag,SECONDMORTGAGERECORDINGDATE,SECONDMORTGAGEDOCDATE,SECONDMORTGAGEDOCUMENTNUMBER,SECONDMORTGAGEDETAILEDDOCUMENTTYPE,SECONDMORTGAGEAMOUNT,SECONDMORTGAGELENDERNAME,SECONDMORTGAGELOANTYPECODE,SECONDMORTGAGETERMCODE,SECONDMORTGAGETERM,SECONDMORTGAGEDUEDATE,SECONDMORTGAGEINTERESTRATETYPE,SECONDMORTGAGEINTERESTRATE,SECONDMORTGAGEEQUITYFLAG,SECONDMORTGAGEOTHERSUBORDINATELOANFlag,LotCode,LandLot,Block,Unit,CityMunicipalityTownship,SubdivisionName,TractNo,SecTwnRngMer,LEGALDESCRIPTION,TITLECOMPANYNAME,MULTIAPN,UPDATETIMESTAMP,UCID
8032,6001,9644734,12-1006-18,Y,4101,NaN,OPAL,ST,NaN,#,4105,OAKLAND,CA,94609.0,2617.0,C014,NaN,BOUTELLE LAURA PH,NaN,NaN,TC,SW,NaN,ZABEL JOSHUA,MM,BOUTELLE,LAURA,PH,NaN,ZABEL,JOSHUA,NaN,NaN,4105,NaN,OPAL,ST,NaN,NaN,NaN,OAKLAND,CA,94609.0,2617.0,C014,1023145716,2009-12-15,2009-12-04,2009386676,36,2,LAWRENCE STACY,NaN,LAWRENCE MARY,NaN,659000,72490,NaN,Y,NaN,NaN,NaN,NaN,2009-12-15,2009-12-04,2.009387e+09,MG,494250,AUSTIN PERRY FINANCIAL CORP,3.0,M,360.0,2040-01-01,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MP,1&2,P,NaN,OAKLAND,VISION OF A PORTION OF CENTRAL OAKLAND T,NaN,NaN,NaN,PLACER TITLE COMPANY,NaN,20161001,U
3398,6001,9667287,2-85-14,NaN,138,NaN,9TH,ST,NaN,APT,1,OAKLAND,CA,94607.0,4776.0,C025,NaN,WONG GANG JUE,NaN,NaN,TC,HW,NaN,WONG XUE XIA PAN,HW,WONG,GANG,JUE,NaN,WONG,XUE,XIA PAN,NaN,138,NaN,9TH,ST,NaN,NaN,NaN,OAKLAND,CA,94607.0,4784.0,C025,1023059352,1996-01-08,1996-01-08,96004870,40,1,WONG GANG JUE,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,0,NaN,NaN,NaN,NaN,NaT,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OAKLAND,NaN,NaN,NaN,NaN,NaN,NaN,20161001,U
3082,6001,9692720,30-1933-16-1,Y,3728,NaN,MAYBELLE,AVE,NaN,A-D,NaN,OAKLAND,CA,94619.0,2142.0,C011,NaN,LAZAR BARBARA L,NaN,NaN,JT,HW,NaN,LAZAR STEVEN,HW,LAZAR,BARBARA,L,NaN,LAZAR,STEVEN,NaN,NaN,3728,NaN,MAYBELLE,AVE,NaN,NaN,NaN,OAKLAND,CA,94619.0,2142.0,C011,1022920426,2009-06-11,2009-06-05,2009184468,40,1,VERNOR SHARON LINFORD,NaN,LINFORD JOHN VALVA,NaN,400000,44000,NaN,NaN,NaN,NaN,NaN,NaN,2009-06-11,2009-06-05,2.009184e+09,MG,110000,SIERRA PACIFIC MORTGAGE COMPANY INC,2.0,M,360.0,2039-07-01,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MP,"51,53&55",A,NaN,OAKLAND,WESTALL TRACT,NaN,NaN,NaN,FIRST AMERICAN TITLE COMPANY,NaN,20161001,U
2563,6001,9718078,40A-3416-39,Y,6919,NaN,LACEY,AVE,NaN,#,B,OAKLAND,CA,94605.0,2579.0,C006,Y,EMERALD PARK HOUSE CORP,NaN,NaN,NaN,CO,NaN,NaN,CO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37968,NaN,CANYON HEIGHTS,DR,NaN,NaN,NaN,FREMONT,CA,94536.0,1861.0,C029,1018810328,2004-06-16

In [42]:
# Location

cols = ['FIPSCODE','PROPERTYUNIQUEID','APN','RECORDINGDATE','HOUSENUMBER','DIRECTIONLEFT','STREETNAME','Suffix','DIRECTIONRIGHT','UNITTYPE',
'UNITNUMBER','CITY','STATE','ZIPCODE']

location = CALtop13.loc[:,cols]

location.head(5)

,FIPSCODE,PROPERTYUNIQUEID,APN,RECORDINGDATE,HOUSENUMBER,DIRECTIONLEFT,STREETNAME,Suffix,DIRECTIONRIGHT,UNITTYPE,UNITNUMBER,CITY,STATE,ZIPCODE
7798,6001,9633057,10-818-22,2008-07-22,520,NaN,OAKLAND,AVE,NaN,NaN,NaN,OAKLAND,CA,94611.0
7164,6001,9633057,10-818-22,2009-05-14,520,NaN,OAKLAND,AVE,NaN,NaN,NaN,OAKLAND,CA,94611.0
5240,6001,9633297,10-821-7,2008-05-23,480,NaN,CHETWOOD,ST,NaN,NaN,NaN,OAKLAND,CA,94610.0
9272,6001,9633300,10-822-10,2010-03-12,474,NaN,JEAN,ST,NaN,NaN,NaN,OAKLAND,CA,94610.0
9273,6001,9633300,10-822-10,2010-03-12,474,NaN,JEAN,ST,NaN,NaN,NaN,OAKLAND,CA,94610.0


In [43]:
example = location.loc[:,:]
example.loc[:,'ZIPCODE'] = example.loc[:,'ZIPCODE'].astype(int)
pd.DataFrame(example).head(5)

,FIPSCODE,PROPERTYUNIQUEID,APN,RECORDINGDATE,HOUSENUMBER,DIRECTIONLEFT,STREETNAME,Suffix,DIRECTIONRIGHT,UNITTYPE,UNITNUMBER,CITY,STATE,ZIPCODE
7798,6001,9633057,10-818-22,2008-07-22,520,NaN,OAKLAND,AVE,NaN,NaN,NaN,OAKLAND,CA,94611
7164,6001,9633057,10-818-22,2009-05-14,520,NaN,OAKLAND,AVE,NaN,NaN,NaN,OAKLAND,CA,94611
5240,6001,9633297,10-821-7,2008-05-23,480,NaN,CHETWOOD,ST,NaN,NaN,NaN,OAKLAND,CA,94610
9272,6001,9633300,10-822-10,2010-03-12,474,NaN,JEAN,ST,NaN,NaN,NaN,OAKLAND,CA,94610
9273,6001,9633300,10-822-10,2010-03-12,474,NaN,JEAN,ST,NaN,NaN,NaN,OAKLAND,CA,94610


In [44]:
example.loc[:,'First'] = example.loc[:,['HOUSENUMBER',
                                        'DIRECTIONLEFT',
                                        'STREETNAME',
                                        'Suffix',
                                        'UNITTYPE',
                                        'UNITNUMBER']].apply(lambda x: ' '.join(x.fillna('').map(str)), axis=1)

example.loc[:,'Last'] = example.loc[:,['STATE',
                                       'ZIPCODE']].apply(lambda x: ' '.join(x.fillna('').map(str)), axis=1)

example.loc[:,'Address'] = example.loc[:,['First','CITY','Last']].apply(lambda x: ', '.join(x.fillna('').map(str)), axis=1)

example.loc[:,'Address'] = example.loc[:,'Address'].str.lower()

example.head(5)

,FIPSCODE,PROPERTYUNIQUEID,APN,RECORDINGDATE,HOUSENUMBER,DIRECTIONLEFT,STREETNAME,Suffix,DIRECTIONRIGHT,UNITTYPE,UNITNUMBER,CITY,STATE,ZIPCODE,First,Last,Address
7798,6001,9633057,10-818-22,2008-07-22,520,NaN,OAKLAND,AVE,NaN,NaN,NaN,OAKLAND,CA,94611,520 OAKLAND AVE,CA 94611,"520 oakland ave , oakland, ca 94611"
7164,6001,9633057,10-818-22,2009-05-14,520,NaN,OAKLAND,AVE,NaN,NaN,NaN,OAKLAND,CA,94611,520 OAKLAND AVE,CA 94611,"520 oakland ave , oakland, ca 94611"
5240,6001,9633297,10-821-7,2008-05-23,480,NaN,CHETWOOD,ST,NaN,NaN,NaN,OAKLAND,CA,94610,480 CHETWOOD ST,CA 94610,"480 chetwood st , oakland, ca 94610"
9272,6001,9633300,10-822-10,2010-03-12,474,NaN,JEAN,ST,NaN,NaN,NaN,OAKLAND,CA,94610,474 JEAN ST,CA 94610,"474 jean st , oakland, ca 94610"
9273,6001,9633300,10-822-10,2010-03-12,474,NaN,JEAN,ST,NaN,NaN,NaN,OAKLAND,CA,94610,474 JEAN ST,CA 94610,"474 jean st , oakland, ca 94610"


In [45]:
CA_TXT_addresses = example['Address']
CA_TXT_addresses = pd.DataFrame(CA_TXT_addresses)
CA_TXT_addresses.head(5)

,Address
7798,"520 oakland ave , oakland, ca 94611"
7164,"520 oakland ave , oakland, ca 94611"
5240,"480 chetwood st , oakland, ca 94610"
9272,"474 jean st , oakland, ca 94610"
9273,"474 jean st , oakland, ca 94610"


In [46]:
# Read in the addresses from rentcrawl to see which can be joined to CA.txt

rentcrawl_addresses = pd.read_csv('rentcrawl_addresses.csv')
rentcrawl_addresses = pd.DataFrame(rentcrawl_addresses.drop('Unnamed: 0',axis=1)) # Remove annoying index
rentcrawl_addresses.columns = ['Address']
rentcrawl_addresses.head(10)

,Address
0,"301 n alvarado st, los angeles, ca 90026"
1,"2656 ellendale pl, los angeles, ca 90007"
2,"470 s mariposa ave, los angeles, ca 90020"
3,"820 s kingsley dr, los angeles, ca 90005"
4,"3935 guardia ave, los angeles, ca 90032"
5,"5906 willoughby ave, los angeles, ca 90038"
6,"821 bridewell st, los angeles, ca 90042"
7,"2626 s orange dr, los angeles, ca 90016"
8,"352 n stanley ave, los angeles, ca 90036"
9,"1326 n alta vista blvd, los angeles, ca 90046"


In [137]:
# Compare the two sets of addresses and print any matches

for i in rentcrawl_addresses.iloc[:,0]:
    if i in CA_TXT_addresses.values:
        print(i)

In [139]:
pd.merge(CA_TXT_addresses,rentcrawl_addresses,on='Address')

,Address


In [46]:
# Select our independent variables to feed into the regression model as 'X'

X = CALtop13 # Be careful here to exclude the SALEAMOUNT variable
X = X.drop('SALEAMOUNT',axis=1)

# Select the dependent variable 'SALEAMOUNT'
Y = CALtop13.SALEAMOUNT

In [49]:
# X.describe(include = 'all').T

In [50]:
# Sort columns by number of missing values
count_nan = len(X) - X.count()
count_nan.sort_values(inplace=True,ascending=False)
count_nan
countDF = pd.DataFrame(count_nan)
# countDF.to_csv('CountDF.csv')

TAXACCOUNTNUMBER                          1350119
AssessorsMapRef                           1350118
LotNumber                                 1350116
District                                  1350111
PARTIALINTERESTINDICATORFLAG              1349854
SECONDMORTGAGERECORDINGBOOK               1349831
SECONDMORTGAGERECORDINGPAGE               1349831
Section                                   1349738
SECONDMORTGAGEINTERESTRATETYPE            1349121
PhaseNo                                   1348799
SECONDMORTGAGEOTHERSUBORDINATELOANFlag    1348380
SECONDMORTGAGETERM                        1348135
SECONDMORTGAGEDUEDATE                     1348135
SECONDMORTGAGETERMCODE                    1348135
MAILUNITNUMBER                            1348038
WAEXCISENUMBER                            1347961
MAILUNITTYPE                              1347911
SECONDMORTGAGEDOCUMENTNUMBER              1347719
SECONDMORTGAGELOANTYPECODE                1347645
NEWCONSTRUCTIONFLAG                       1347590


In [54]:
city_count = pd.DataFrame(pd.crosstab(index=X.CITY,columns="Count"))
city_count = pd.DataFrame(city_count['Count'].sort_values(ascending=False))
city_count.head(6)

,Count
CITY,
LOS ANGELES,200112
SAN FRANCISCO,77088
SAN DIEGO,53507
LONG BEACH,41605
OAKLAND,33304
SACRAMENTO,28792


In [55]:
X

,FIPSCODE,PROPERTYUNIQUEID,APN,TAXACCOUNTNUMBER,MatchedFlag,HOUSENUMBER,DIRECTIONLEFT,STREETNAME,Suffix,DIRECTIONRIGHT,UNITTYPE,UNITNUMBER,CITY,STATE,ZIPCODE,ZIPPLUS4,CARRIERCODE,CORPORATEINDICATOR,BUYERNAMEBORROWERNAME1,OWNERETALIND,OwnerCAREOfNAME,OWNERSHIPRIGHTSCODE1,OWNERRELATIONSHIPTYPE1,CORPORATEINDICATOR2,BUYERNAMEBORROWERNAME2,OWNERSHIPRIGHTSCODE2,OWNERRELATIONSHIPTYPE2,BUYER1LASTNAME,BUYER1FIRSTNAME,BUYER1MIDDLEName,BUYER1SUFFIX,BUYER2LASTNAME,BUYER2FIRSTNAME,BUYER2MIDDLEName,BUYER2SUFFIX,MAILHOUSENUMBER,MAILDIRECTIONLEFT,MAILSTREETNAME,MAILMODE,MAILDIRECTIONRIGHT,MAILUNITTYPE,MAILUNITNUMBER,MAILCITY,MAILSTATE,MAILZIPCODE,MAILZIPPlus4,MAILCARRIERCODE,FirstAmericanUniqueTransactionID,RECORDINGDATE,SALEDATE,DOCUMENTNUMBER,RECORDINGBOOK,RECORDINGPAGE,WAEXCISENUMBER,DOCUMENTTYPE,TRANSACTIONTYPE,SELLERNAME,SELLERNAMECORPINDICATOR,SELLERNAME2,SELLERNAME2CORPINDICATOR,TRANSFERTAX,PARTIALINTERESTINDICATORFLAG,RESALEFLAG,NEWCONSTRUCTIONFLAG,INTERFAMILYFLAG,FORCLOSUREFLAG,REOSALEFLAG,FIRSTMORTGAGERECORDINGDATE,FIRSTMORTGAGEDOCDATE,FIRSTMORTGAGEDOCUMENTNUMBER,FIRSTMORTGAGERECORDINGBOOK,FIRSTMORTGAGERECORDINGPAGE,FIRSTMORTGAGEDETAILEDDOCUMENTTYPE,FIRSTMORTGAGEMORTGAGEAMOUNT,FIRSTMORTGAGELENDERNAME,FIRSTMORTGAGELOANTYPECODE,FIRSTMORTGAGETERMCODE,FIRSTMORTGAGETERM,FIRSTMORTGAGEDUEDATE,FIRSTMORTGAGEINTERESTRATETYPE,FIRSTMORTGAGEINTERESTRATE,FIRSTMORTGAGESELLERCARRYBACKFLAG,FIRSTMORTGAGEPRIVATEPARTYLENDER,FIRSTMORTGAGECONSTRUCTIONLOAN,FIRSTMORTGAGEREFIFLAG,FIRSTMORTGAGEEQUITYFLAG,FIRSTMORTGAGEHELOCFLAG,FIRSTMORTGAGEOTHERSUBORDINATELOANFlag,SECONDMORTGAGERECORDINGDATE,SECONDMORTGAGEDOCDATE,SECONDMORTGAGEDOCUMENTNUMBER,SECONDMORTGAGERECORDINGBOOK,SECONDMORTGAGERECORDINGPAGE,SECONDMORTGAGEDETAILEDDOCUMENTTYPE,SECONDMORTGAGEAMOUNT,SECONDMORTGAGELENDERNAME,SECONDMORTGAGELOANTYPECODE,SECONDMORTGAGETERMCODE,SECONDMORTGAGETERM,SECONDMORTGAGEDUEDATE,SECONDMORTGAGEINTERESTRATETYPE,SECONDMORTGAGEINTERESTRATE,SECONDMORTGAGEEQUITYFLAG,SECONDMORTGAGEOTHERSUBORDINATELOANFlag,LotCode,LotNumber,LandLot,Block,Section,District,Unit,CityMunicipalityTownship,SubdivisionName,PhaseNo,TractNo,SecTwnRngMer,AssessorsMapRef,LEGALDESCRIPTION,TITLECOMPANYNAME,MULTIAPN,UPDATETIMESTAMP,UCID
0,6001,9927865,70-152-45,NaN,Y,1336,NaN,BROADWAY,NaN,NaN,NaN,NaN,ALAMEDA,CA,94501.0,4651.0,C027,NaN,STOHR MARTIN W,3,NaN,TC,TR,NaN,BOWEN PATRICIA L,NaN,TR,STOHR,MARTIN,W,NaN,BOWEN,PATRICIA,L,NaN,1033,NaN,POST,ST,NaN,NaN,NaN,ALAMEDA,CA,94501,5530.0,C003,1.022176e+09,20080404.0,20080401.0,2008112955,NaN,NaN,NaN,40.0,1.0,STOHR MARTIN W,NaN,STOHR TRUST,Y,0.0,NaN,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,P,NaN,16,NaN,NaN,NaN,NaN,ALAMEDA,HALF ACRE LOTS IN THE HAYS & CAPERTON TR,NaN,NaN,NaN,NaN,NaN,NONE AVAILABLE,NaN,20160927.0,U
1,6001,9926228,70-147-38,NaN,Y,1118,NaN,BROADWAY,NaN,NaN,NaN,NaN,ALAMEDA,CA,94501.0,7756.0,C010,NaN,STOHR MARTIN W,3,NaN,TC,TR,NaN,BOWEN PATRICIA L,NaN,TR,STOHR,MARTIN,W,NaN,BOWEN,PATRICIA,L,NaN,1033,NaN,POST,ST,NaN,NaN,NaN,ALAMEDA,CA,94501,5530.0,C003,1.022176e+09,20080404.0,20080401.0,2008112957,NaN,NaN,NaN,40.0,1.0,STOHR MARTIN W,NaN,STOHR TRUST,Y,0.0,NaN,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,NaN,NaN,NaN,NaN,ALAMEDA,SUBDIVISION OF HALF ACRE LOTS HAYS & CAP,NaN,NaN,NaN,NaN,SUBDIVISION #2,NONE AVAILABLE,NaN,20160927.0,U
2,6001,9666761,26-768-18,NaN,Y,3048,NaN,LOGAN,ST,NaN,NaN,NaN,OAKLAND,CA,94601.0,1927.0,C003,NaN,MAN LAP PUI,NaN,NaN,JT,HW,NaN,MAN YUET SING,NaN,HW,MAN,LAP,PUI,NaN,MAN,YUET,SING,NaN,1115,NaN,GARFIELD,AVE,NaN,NaN,NaN,ALBANY,CA,94706,1230.0,C074,1.019521e+09,19951113.0,19951106.0,95265344,NaN,NaN,NaN,39.0,2.0,SUE PIK WAN,NaN,SUE QUING WONG,NaN,15400.0,NaN,Y,NaN,NaN,NaN,NaN,19951113.0,19951106.0,NaN,NaN,NaN,MG,98000.0,AMERICAN SVG BANK,3,M,180.0,2.01012e+07,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,

In [22]:
# Perform scaling and dummy coding for all X variables
X.head(5)


# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# Use dummy variables for categoricals so we can standardize the data
one_hot_encoded_training_predictors = pd.get_dummies(X_train)
one_hot_encoded_test_predictors = pd.get_dummies(X_test)

# Ensure that the column names and column order are equivalent in both sets
final_train, final_test = one_hot_encoded_training_predictors.align(one_hot_encoded_test_predictors,
                                                                    join='left', 
                                                                    axis=1)

,FIPSCODE,PROPERTYUNIQUEID,APN,TAXACCOUNTNUMBER,MatchedFlag,HOUSENUMBER,DIRECTIONLEFT,STREETNAME,Suffix,DIRECTIONRIGHT,UNITTYPE,UNITNUMBER,CITY,STATE,ZIPCODE,ZIPPLUS4,CARRIERCODE,CORPORATEINDICATOR,BUYERNAMEBORROWERNAME1,OWNERETALIND,OwnerCAREOfNAME,OWNERSHIPRIGHTSCODE1,OWNERRELATIONSHIPTYPE1,CORPORATEINDICATOR2,BUYERNAMEBORROWERNAME2,OWNERSHIPRIGHTSCODE2,OWNERRELATIONSHIPTYPE2,BUYER1LASTNAME,BUYER1FIRSTNAME,BUYER1MIDDLEName,BUYER1SUFFIX,BUYER2LASTNAME,BUYER2FIRSTNAME,BUYER2MIDDLEName,BUYER2SUFFIX,MAILHOUSENUMBER,MAILDIRECTIONLEFT,MAILSTREETNAME,MAILMODE,MAILDIRECTIONRIGHT,MAILUNITTYPE,MAILUNITNUMBER,MAILCITY,MAILSTATE,MAILZIPCODE,MAILZIPPlus4,MAILCARRIERCODE,FirstAmericanUniqueTransactionID,RECORDINGDATE,SALEDATE,DOCUMENTNUMBER,RECORDINGBOOK,RECORDINGPAGE,WAEXCISENUMBER,DOCUMENTTYPE,TRANSACTIONTYPE,SELLERNAME,SELLERNAMECORPINDICATOR,SELLERNAME2,SELLERNAME2CORPINDICATOR,TRANSFERTAX,PARTIALINTERESTINDICATORFLAG,RESALEFLAG,NEWCONSTRUCTIONFLAG,INTERFAMILYFLAG,FORCLOSUREFLAG,REOSALEFLAG,FIRSTMORTGAGERECORDINGDATE,FIRSTMORTGAGEDOCDATE,FIRSTMORTGAGEDOCUMENTNUMBER,FIRSTMORTGAGERECORDINGBOOK,FIRSTMORTGAGERECORDINGPAGE,FIRSTMORTGAGEDETAILEDDOCUMENTTYPE,FIRSTMORTGAGEMORTGAGEAMOUNT,FIRSTMORTGAGELENDERNAME,FIRSTMORTGAGELOANTYPECODE,FIRSTMORTGAGETERMCODE,FIRSTMORTGAGETERM,FIRSTMORTGAGEDUEDATE,FIRSTMORTGAGEINTERESTRATETYPE,FIRSTMORTGAGEINTERESTRATE,FIRSTMORTGAGESELLERCARRYBACKFLAG,FIRSTMORTGAGEPRIVATEPARTYLENDER,FIRSTMORTGAGECONSTRUCTIONLOAN,FIRSTMORTGAGEREFIFLAG,FIRSTMORTGAGEEQUITYFLAG,FIRSTMORTGAGEHELOCFLAG,FIRSTMORTGAGEOTHERSUBORDINATELOANFlag,SECONDMORTGAGERECORDINGDATE,SECONDMORTGAGEDOCDATE,SECONDMORTGAGEDOCUMENTNUMBER,SECONDMORTGAGERECORDINGBOOK,SECONDMORTGAGERECORDINGPAGE,SECONDMORTGAGEDETAILEDDOCUMENTTYPE,SECONDMORTGAGEAMOUNT,SECONDMORTGAGELENDERNAME,SECONDMORTGAGELOANTYPECODE,SECONDMORTGAGETERMCODE,SECONDMORTGAGETERM,SECONDMORTGAGEDUEDATE,SECONDMORTGAGEINTERESTRATETYPE,SECONDMORTGAGEINTERESTRATE,SECONDMORTGAGEEQUITYFLAG,SECONDMORTGAGEOTHERSUBORDINATELOANFlag,LotCode,LotNumber,LandLot,Block,Section,District,Unit,CityMunicipalityTownship,SubdivisionName,PhaseNo,TractNo,SecTwnRngMer,AssessorsMapRef,LEGALDESCRIPTION,TITLECOMPANYNAME,MULTIAPN,UPDATETIMESTAMP,UCID
0,6001,9927865,70-152-45,NaN,Y,1336,NaN,BROADWAY,NaN,NaN,NaN,NaN,ALAMEDA,CA,94501.0,4651.0,C027,NaN,STOHR MARTIN W,3,NaN,TC,TR,NaN,BOWEN PATRICIA L,NaN,TR,STOHR,MARTIN,W,NaN,BOWEN,PATRICIA,L,NaN,1033,NaN,POST,ST,NaN,NaN,NaN,ALAMEDA,CA,94501,5530.0,C003,1.022176e+09,20080404.0,20080401.0,2008112955,NaN,NaN,NaN,40.0,1.0,STOHR MARTIN W,NaN,STOHR TRUST,Y,0.0,NaN,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,P,NaN,16,NaN,NaN,NaN,NaN,ALAMEDA,HALF ACRE LOTS IN THE HAYS & CAPERTON TR,NaN,NaN,NaN,NaN,NaN,NONE AVAILABLE,NaN,20160927.0,U
1,6001,9926228,70-147-38,NaN,Y,1118,NaN,BROADWAY,NaN,NaN,NaN,NaN,ALAMEDA,CA,94501.0,7756.0,C010,NaN,STOHR MARTIN W,3,NaN,TC,TR,NaN,BOWEN PATRICIA L,NaN,TR,STOHR,MARTIN,W,NaN,BOWEN,PATRICIA,L,NaN,1033,NaN,POST,ST,NaN,NaN,NaN,ALAMEDA,CA,94501,5530.0,C003,1.022176e+09,20080404.0,20080401.0,2008112957,NaN,NaN,NaN,40.0,1.0,STOHR MARTIN W,NaN,STOHR TRUST,Y,0.0,NaN,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,NaN,NaN,NaN,NaN,ALAMEDA,SUBDIVISION OF HALF ACRE LOTS HAYS & CAP,NaN,NaN,NaN,NaN,SUBDIVISION #2,NONE AVAILABLE,NaN,20160927.0,U
2,6001,9666761,26-768-18,NaN,Y,3048,NaN,LOGAN,ST,NaN,NaN,NaN,OAKLAND,CA,94601.0,1927.0,C003,NaN,MAN LAP PUI,NaN,NaN,JT,HW,NaN,MAN YUET SING,NaN,HW,MAN,LAP,PUI,NaN,MAN,YUET,SING,NaN,1115,NaN,GARFIELD,AVE,NaN,NaN,NaN,ALBANY,CA,94706,1230.0,C074,1.019521e+09,19951113.0,19951106.0,95265344,NaN,NaN,NaN,39.0,2.0,SUE PIK WAN,NaN,SUE QUING WONG,NaN,15400.0,NaN,Y,NaN,NaN,NaN,NaN,19951113.0,19951106.0,NaN,NaN,NaN,MG,98000.0,AMERICAN SVG BANK,3,M,180.0,2.01012e+07,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,